## Show Comparator

### Assignment
**Deliverable:** Author a segment of an ETL pipeline that will ingest or process raw data.
You must also submit a URL to a GitHub repository for your solution. In python you’ll
need to know how to open files, iterate files, pattern match and output files.

**Benchmarks:**
1. Your data processor should be able to ingest a pre-defined data source and
perform at least three of these operations:
    * Fetch / download / retrieve a remote data file by URL (API call like we did
    in class), or ingest a local file that you have downloaded from
    somewhere…like in a CSV format. Suggestions for remote data sources
    are listed at the end of this document on our Github page as we went
    through in class.
    * Convert the general format and data structure of the data source (from
    JSON to CSV, from CSV to JSON, from JSON into a SQL database table,
    etc. I want the option to convert any source to any target. So, if I get a
    CSV as an input, I want the user to choose an output)
    * Modify the number of columns from the source to the destination,
    reducing or adding columns so that you transform it with something
    else…you can make up whatever it is…like date changes…or mash up
    two columns…it’s up to you.
    * The converted (new) file should be written to disk (local file) or written to a
SQL database like SQL Lite
    * Generate a brief summary of the data file ingestion after it has processed
and output it to the user including:
        * Number of records
        * Number of columns
2. The processor should produce informative errors should it be unable to complete
an operation. (Try / Catch with error messages, not file exists…just pick any
error.)

### About the code
One function was created for user interaction while the other two are helper methods 
1. Utilizes the [TVMaze API](https://www.tvmaze.com/api#show-lookup) to query data on TV Shows
2. Filters API data to only include *Genre* and *rating*
3. Outputs data in sql database, csv file, or none depending on selection
4. Sorts shows on most highly rated

### How to use the Code
Call the main method: ```get_api_response()``` which has four parameters
1. First show name (case insensitive)
2. Second show name (case insensitive)
3. Third show name (case insensitive)
4. Output file type -- this can be either ```csv```, ```sql```, or ```None``` (case insensitive)

Call the visualization method: ```visualize_show()``` which also has four parameters
1. First show name (same as from main method call)
2. Second show name (same as from main method call)
3. Third show name (same as from main method call)
4. Graph Type -- this can be either ```'box'```, ```'violin'```, ```'scatter'```, ```'line'```

In [11]:
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd
import sqlite3
import plotly as plt

In [12]:
# Function for calling API to get data from desired show
def __query_response__(show_name, url):
    header_var ={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    show_querystring = {'q': show_name, 'code': 0} #type dict
    # Writing exceptions for if the url is not working
    try:
        show_response = requests.request("GET", url, headers=header_var, params=show_querystring)
        show_response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        return f"This show '{show_name}' does not exist: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "Connection" + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "Timeout error please try again: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "f4: " + repr(err)
    json = show_response.json()
    num_keys = len(json[0]['show'])
    num_responses = len(json)
    print(f'{show_name} has: {num_keys} columns & {num_responses} records')
    return json

# Functin to invoke SQLite3 commands to create a local .db file from desired data
def __make_sql_db__(df):
    try:
        conn = sqlite3.connect('showdata.db')
        cur = conn.cursor()
        cur.execute('CREATE TABLE IF NOT EXISTS shows (Show_Name text, Genres text, Ratings number)')
        conn.commit()
    except sqlite3.Error as er:
        print('SQLite error: %s' % (' '.join(er.args)))
        print("Exception class is: ", er.__class__)
    df.to_sql('shows', conn, if_exists='replace', index=False) 
    cur.execute('''  
    SELECT * FROM shows
        ''')
    for row in cur.fetchall():
        print (row)
    return "Added to SQL database"
    conn.close()

# function which collects three show names, and a desired output type to return name of show, genres, and ratings in either a SQL db or CSV
def get_api_response(show_1, show_2, show_3, response_type):
    # Getting url for the financeData module in Yahoo API
    url = 'http://api.tvmaze.com/search/shows?'
    
    show1_json = __query_response__(show_1, url)
    
    if len(show1_json)>0:
        show1_name = show1_json[0]['show']['name']
        show1_genrelist = show1_json[0]['show']['genres']
        show1_rating = show1_json[0]['show']['rating']['average']
        show1_imdb = show1_json[0]['show']['externals']['imdb']
    else:
        return "No data recieved -- invalid show name"
       
    show2_json = __query_response__(show_2, url)
    if len(show2_json) > 0:
        show2_name = show2_json[0]['show']['name']
        show2_genrelist = show2_json[0]['show']['genres']
        show2_rating = show2_json[0]['show']['rating']['average']
        show2_imdb = show2_json[0]['show']['externals']['imdb']
        #imdb_show12_url = 
    else:
        return "No data recieved -- invalid show name"

    
    show3_json = __query_response__(show_3, url)
    if len(show3_json) > 0:
        show3_name = show3_json[0]['show']['name']
        show3_genrelist = show3_json[0]['show']['genres']
        show3_rating = show3_json[0]['show']['rating']['average']
        show3_imdb = show3_json[0]['show']['externals']['imdb']
    else:
        return "No data recieved -- invalid show name"
    
    namelist = [show1_name, show2_name, show3_name] #list of show names
    genrelist = ['|'.join(show1_genrelist), '|'.join(show2_genrelist), '|'.join(show3_genrelist)] #list of strings of genres "pipe seperated values"
    ratinglist = [show1_rating, show2_rating, show3_rating] #list of ratings 

    #making dataframe from list of values using zip method
    df = pd.DataFrame(list(zip(namelist,genrelist, ratinglist)),
               columns =['Show_Name','Genres', 'Ratings']).sort_values('Ratings', ascending=False) #sorts values from highest rating to lowest rating
    response_type = response_type.lower()
    if response_type == 'csv':
        df.to_csv('showdata.csv')
        result = "File Written to CSV"
    elif response_type == 'sql':
        result = __make_sql_db(df)
    elif response_type == 'none':
        result = None
    else:
        result = "An unhandled error has occurred!"
    print(result)
    return df

### Run the above box and then tweak selections below

In [13]:
#Adjust below code to desired outputs
show_1 = 'Breaking Bad' 
show_2 = 'New Girl'
show_3 = 'Keeping up with the Kardashians'
output_type = 'csv'

#Leave code below unchanged
get_api_response(show_1, show_2, show_3, output_type)

Breaking Bad has: 23 columns & 7 records
New Girl has: 23 columns & 5 records
Keeping up with the Kardashians has: 23 columns & 1 records
File Written to CSV


,Show_Name,Genres,Ratings
0,Breaking Bad,Drama|Crime|Thriller,9.2
1,New Girl,Comedy|Romance,7.2
2,Keeping Up with the Kardashians,Drama,3.2


## Visualizing your Results

Use the ```visualize_show()``` method to create a nice visualization to depict difference in show 

In [14]:
import plotly.express as px
import plotly.graph_objects as go

In [28]:
def visualize_show(show_1, show_2, show_3, graph_type):
    df = get_api_response(show_1, show_2, show_3, 'None')
    plot_dict = {'box': px.box,'violin': px.violin, 'scatter': px.scatter, 'line':px.line} 
    try:
        # Initialize function
        fig = plot_dict[graph_type](df, 
                                    x='Show_Name', 
                                    y='Ratings',
                                   hover_name = "Show_Name")
        # Format figure 
        title_string = f'Chart: {graph_type} plot of Show Name and Rating'
        fig.update_layout(title = title_string)
        fig.update_xaxes(tickangle=-45)
        return fig
    
    except KeyError:
        print("Key not found. Make sure that 'graph_type' is in ['box','violin', 'scatter', 'line']")

In [32]:
#Adjust below code to desired selection from the following selection ['box','violin', 'scatter', 'line']
graph_type = 'violin'

#Leave code below unchanged
visualize_show(show_1, show_2, show_3, graph_type)

Breaking Bad has: 23 columns & 7 records
New Girl has: 23 columns & 5 records
Keeping up with the Kardashians has: 23 columns & 1 records
None
